In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import math
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, TargetEncoder
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, train_test_split, HalvingRandomSearchCV, HalvingGridSearchCV
from sklearn.compose import make_column_transformer, TransformedTargetRegressor, make_column_selector
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.inspection import permutation_importance

# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

According to the article, the dataset represents 17 marketing campaigns, which "occurred between May 2008 and November 2010, corresponding to a total of 79354 contacts."

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

From the information above, we can make the following observations:
- There are no missing data
- For the purposes of creating a realistic predictive model, the column "duration" should be dropped because this variable was not known before the call is made and is correlated with the target variable.
- One Hot Encoder transform should be applied to all "object" type data
- Numerical data should be scaled

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

The bank aims to invest in a direct campaign by using predictive modeling to identify high-quality potential customers. This targeted approach is preferrable to mass campaigns as it results in more efficient allocation of resources.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [6]:
# Drop columns from X
X = df.drop(['duration', 'y'], axis = 1)

# Convert y to boolean values
y = df['y'] == 'yes'

In [7]:
# Transformer to be used in pipeline
tf = make_column_transformer((OneHotEncoder(drop = 'if_binary'), make_column_selector(dtype_include=object)),
                              remainder = StandardScaler(),
                              verbose_feature_names_out = False)

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, shuffle = True)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

We will establish a baseline model which always returns the majority class result:

In [9]:
y_train.value_counts()

y
False    27413
True      3478
Name: count, dtype: int64

In [10]:
y_train.value_counts(normalize = True)

y
False    0.887411
True     0.112589
Name: proportion, dtype: float64

We can see that the majority class is "no" which occurs with probability 0.886763. This will be the baseline our classifier will try to beat.

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [11]:
lr_pipe = Pipeline([('tf', tf),
              ('lr', LogisticRegression(max_iter = 1000000))])

In [12]:
lr_pipe.fit(X_train, y_train)

Pipeline(steps=[('tf',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x124ccb290>)],
                                   verbose_feature_names_out=False)),
                ('lr', LogisticRegression(max_iter=1000000))])

### Problem 9: Score the Model

What is the accuracy of your model?

In [13]:
print('The training accuracy of the logistic regression model is:', accuracy_score(y_train, lr_pipe.predict(X_train)))

The training accuracy of the logistic regression model is: 0.8999384934123208


In [14]:
print('The testing accuracy of the logistic regression model is:', accuracy_score(y_test, lr_pipe.predict(X_test)))

The testing accuracy of the logistic regression model is: 0.9005535592891133


We see that the logistic regression classifier only performs a few percentages better than the baseline model.

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [15]:
# Function to calculate accuracy, precision, and recall scores
def get_scores(classifier):
    if classifier == 'LogisticRegression':
        kwargs = {'max_iter': 1000000}
    else:
        kwargs = {}
    pipe = Pipeline([
                    ('tf', tf),
                    ('classifier', eval(classifier)(**kwargs))])
    tic = time.time()
    pipe.fit(X_train, y_train)
    toc = time.time() - tic
    train_acc = accuracy_score(y_train, pipe.predict(X_train))
    test_acc = accuracy_score(y_test, pipe.predict(X_test))
    train_prec = precision_score(y_train, pipe.predict(X_train))
    test_prec = precision_score(y_test, pipe.predict(X_test))
    train_rec = recall_score(y_train, pipe.predict(X_train))
    test_rec = recall_score(y_test, pipe.predict(X_test))
    scores_dict = {"Model": classifier,
                  "Train Time": toc,
                  "Train Accuracy": train_acc,
                  "Test Accuracy": test_acc,
                  "Train Precision": train_prec,
                  "Test Precision": test_prec,
                  "Train Recall": train_prec,
                  "Test Recall": test_prec}
    return scores_dict

In [16]:
# List of classifiers to use
classifier_list = ['LogisticRegression',
                   'KNeighborsClassifier',
                   'DecisionTreeClassifier',
                   'SVC']

In [17]:
results_df = pd.DataFrame([get_scores(classifier) for classifier in classifier_list], index = list(range(len(classifier_list))))

In [18]:
results_df

,Model,Train Time,Train Accuracy,Test Accuracy,Train Precision,Test Precision,Train Recall,Test Recall
0,LogisticRegression,1.654603,0.899938,0.900554,0.663844,0.656109,0.663844,0.656109
1,KNeighborsClassifier,0.148265,0.912434,0.889288,0.707239,0.516717,0.707239,0.516717
2,DecisionTreeClassifier,0.526192,0.995565,0.838594,0.998508,0.306801,0.998508,0.306801
3,SVC,166.784294,0.903953,0.902399,0.697602,0.667377,0.697602,0.667377


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?

There was no gender feature provided in this dataset, however we can take a look at the logistic regression coefficients to see which features had the most impact on the predictive model

In [19]:
# Get features coef list
coef_dict = dict(zip(lr_pipe['tf'].get_feature_names_out(), lr_pipe['lr'].coef_[0]))
coef_dict = {k: v for k, v in sorted(coef_dict.items(), key=lambda item: item[1], reverse = True)}
coef_importance_df = pd.DataFrame(coef_dict.values(), index = coef_dict.keys(), columns = ['Coefficient'])

In [20]:
coef_importance_df.head()

,Coefficient
month_mar,1.313275
education_illiterate,0.923076
cons.price.idx,0.905687
poutcome_success,0.398592
euribor3m,0.380918


We can also use permutation importance:

In [21]:
perm_imp = permutation_importance(lr_pipe, X_test, y_test, n_repeats = 10)

In [22]:
permimp_dict = dict(zip(X.columns, perm_imp.importances_mean))
permimp_df = pd.DataFrame(permimp_dict.values(), index = permimp_dict.keys(), columns = ['Permutation Importance'])
permimp_df = permimp_df.sort_values(by = 'Permutation Importance', ascending = False)
permimp_df.head()

,Permutation Importance
emp.var.rate,0.160901
cons.price.idx,0.035505
euribor3m,0.010615
pdays,0.007662
poutcome,0.002360


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree. Adjust your performance metric

In [23]:
# Define parameters to grid search over
param_grid_dict = {'LogisticRegression': {'classifier__C': [0.1, 1, 10]},
                   'KNeighborsClassifier': {'classifier__n_neighbors': [4, 6, 8],
                                            'classifier__weights': ['uniform', 'distance']},
                   'DecisionTreeClassifier': {'classifier__max_depth': [None, 10, 30],
                                              'classifier__min_samples_split': [3, 5, 7],
                                              'classifier__class_weight': [None, 'balanced']},
                   'SVC': {'classifier__C': [0.1, 1, 10],
                           'classifier__kernel': ['linear', 'rbf', 'poly']}
}

In [24]:
te = make_column_transformer((TargetEncoder(target_type = 'binary'), 
                              make_column_selector(dtype_include=object)),
                              remainder = StandardScaler(),
                              verbose_feature_names_out = False)

In [25]:
def get_grid_scores(classifier, param_grid, scoring):
    if classifier == 'LogisticRegression':
        kwargs = {'max_iter': 1000000}
    else:
        kwargs = {}
    pipe = Pipeline([
                    ('tf', te),
                    ('classifier', eval(classifier)(**kwargs))])
    grid = HalvingGridSearchCV(estimator = pipe,
                        param_grid = param_grid,
                        scoring = scoring)
    tic = time.time()
    grid.fit(X_train, y_train)
    toc = time.time() - tic
    train_acc = accuracy_score(y_train, grid.predict(X_train))
    test_acc = accuracy_score(y_test, grid.predict(X_test))
    train_prec = precision_score(y_train, grid.predict(X_train))
    test_prec = precision_score(y_test, grid.predict(X_test))
    train_rec = recall_score(y_train, grid.predict(X_train))
    test_rec = recall_score(y_test, grid.predict(X_test))
    scores_dict = {"Model": classifier,
                  "Train Time": toc,
                  "Train Accuracy": train_acc,
                  "Test Accuracy": test_acc,
                  "Train Precision": train_prec,
                  "Test Precision": test_prec,
                  "Train Recall": train_prec,
                  "Test Recall": test_prec}
    return scores_dict

In [26]:
grid_scores_df = pd.DataFrame([get_grid_scores(classifier = classifier,
                                               param_grid = param_grid_dict[classifier],
                                               scoring = 'accuracy')
                              for classifier in param_grid_dict.keys()], 
                              index = list(range(len(param_grid_dict))))

In [27]:
grid_scores_df

,Model,Train Time,Train Accuracy,Test Accuracy,Train Precision,Test Precision,Train Recall,Test Recall
0,LogisticRegression,5.535712,0.899097,0.900942,0.684749,0.689840,0.684749,0.689840
1,KNeighborsClassifier,40.706017,0.905312,0.897640,0.712856,0.619469,0.712856,0.619469
2,DecisionTreeClassifier,13.175140,0.909391,0.893658,0.708155,0.558261,0.708155,0.558261
3,SVC,18371.115077,0.896669,0.900068,0.627907,0.667506,0.627907,0.667506


## Analysis

- Overall, all 4 of the models were able to achieve an accuracy of about 90%
- However, looking at the baseline model of guessing the majority class which has an accuracy of 88.7%, our models did not exceed the baseline by very much
- Our data was very unbalanced which resulted in poor precision and recall scores
- Recommend using balanced classifiers
- Can possiby reduce training data to ~top 5 features from permutation importance data with minimal reduction in accuracy

## Best Classifier - Logistic Regression
- Comparing results using default parameters vs grid search, we do not notice much of an increase in accuracy
- Performed best overall across all metric
- Minimal computation time

## Worst Classifier - SVC
- Did not see a drastic improvement to scores compared to other classifiers
- Computationally expensive -takes about 100x longer